In [1]:
import os
import argparse
import time
from datetime import datetime, date
import random

import numpy as np
from scipy.sparse import load_npz
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve, auc
from scipy.stats import pearsonr
import pandas as pd

import torch
import torch_geometric
import torch.nn.functional as F
import torch.nn as nn

from sage_conv_cat_ import SAGEConvCat

def to_cpu_npy(x):
    return x.cpu().detach().numpy()

ModuleNotFoundError: No module named 'torch_sparse'

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_classification(nn.Module):

    def __init__(self, num_feat, num_conv_layers, conv_layer_sizes, num_lin_layers, lin_hidden_sizes, num_classes):
        '''
        Defines regression model class

        Parameters
        ----------
        num_feat [int]: Feature dimension (int)
        num_conv_layers [int]: Number of convolutional layers (1, 2, or 3)
        conv_layer_sizes [list]: Embedding size of convolutional layers 
        num_lin_layers [int]: Number of linear layers (1, 2, or 3)
        lin_hidden_sizes [list]: Embedding size of hidden linear layers
        num_classes [int]: Size of predicted output tensor for batch size of N, 
            i.e. N x num_classes(=1)

        Returns
        -------
        None.

        '''
        
        super(CNN_classification, self).__init__()
        
        self.num_conv_layers = num_conv_layers
        self.num_lin_layers = num_lin_layers
        self.dropout = 0.5
    
        # Define convolutional layers
        if self.num_conv_layers >= 1:
            self.conv1 = nn.Conv1d(num_feat, conv_layer_sizes[0], kernel_size=3)
        if self.num_conv_layers >= 2:
            self.conv2 = nn.Conv1d(conv_layer_sizes[0], conv_layer_sizes[1], kernel_size=3)
        if self.num_conv_layers >= 3:
            self.conv3 = nn.Conv1d(conv_layer_sizes[1], conv_layer_sizes[2], kernel_size=3)
        
        # Define linear layers
        if self.num_lin_layers >= 1:
            self.lin1 = nn.Linear(lin_hidden_sizes[0], lin_hidden_sizes[1])
        if self.num_lin_layers >= 2:
            self.lin2 = nn.Linear(lin_hidden_sizes[1], lin_hidden_sizes[2])
        if self.num_lin_layers == 3:
            self.lin3 = nn.Linear(lin_hidden_sizes[2], lin_hidden_sizes[3])
            
        self.loss_calc = nn.CrossEntropyLoss()
        self.torch_softmax = nn.Softmax(dim=1)

        
    def forward(self, x, train_status=False):
        '''
        Forward function

        Parameters
        ----------
        x [tensor]: Node features
        edge_index [tensor]: Subgraph mask
        train_status [bool]: optional, set to True for dropout

        Returns
        -------
        scores [tensor]: Predicted expression levels
        '''

        ### Graph convolution module
        if self.num_conv_layers == 1:
            h = self.conv1(x, edge_index)
            h = torch.relu(h)
        elif self.num_conv_layers == 2:
            h = self.conv1(x, edge_index)
            h = torch.relu(h)
            h = self.conv2(h, edge_index)
            h = torch.relu(h)
        elif self.num_conv_layers == 3:
            h = self.conv1(x, edge_index)
            h = torch.relu(h)
            h = self.conv2(h, edge_index)
            h = torch.relu(h)
            h = self.conv3(h, edge_index)
            h = torch.relu(h)
            
        h = F.dropout(h, p = self.dropout_value, training=train_status)

        ### Linear module
        if self.num_lin_layers == 1:
            scores = self.lin1(h)
        elif self.num_lin_layers == 2:
            scores = self.lin1(h)
            scores = torch.relu(scores)
            scores = self.lin2(scores)
        elif self.num_lin_layers == 3:
            scores = self.lin1(h)
            scores = torch.relu(scores)
            scores = self.lin2(scores)
            scores = torch.relu(scores)
            scores = self.lin3(scores)
        
        return scores

    
    def loss(self, scores, targets):
        '''
        Calculates mean squared error loss
        
        Parameters
        ----------
        scores [tensor]: Predicted scores from forward function
        targets [tensor]: Target scores 

        Returns
        -------
        mse [tensor]: Mean squared error loss

        '''
        
        mse = self.loss_calc(scores, targets)

        return mse


In [49]:
def train_model_cnn_classification(model, X, y, max_epoch, learning_rate, train_idx, valid_idx, optimizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X = X.to(device)
    y = y.to(device)
    optimizer = optimizer

    train_labels = y[train_idx].cpu().numpy()
    valid_labels = y[valid_idx].cpu().numpy()

    train_loss_list = []
    train_AUROC_vec = np.zeros(np.shape(np.arange(max_epoch)))
    valid_loss_list = []
    valid_AUROC_vec = np.zeros(np.shape(np.arange(max_epoch)))

    model.train()
    train_status = True
    print('\n')
    for e in range(max_epoch):
        if e % 100 == 0:
            print(f"Epoch {e} out of {max_epoch}")
        
        model.train()
        train_status = True
        optimizer.zero_grad()

        forward_scores = model(X, train_status)
        train_scores = forward_scores[train_idx]
        train_loss = model.loss(train_scores, y[train_idx])
        train_softmax, _ = model.calc_softmax_pred(train_scores)
        
        train_loss.backward()
        optimizer.step()

        model.eval()
        valid_scores = forward_scores[valid_idx]
        valid_loss = model.loss(valid_scores, y[valid_idx])
        valid_softmax, _ = model.calc_softmax_pred(valid_scores)

        train_loss_list.append(train_loss.item())
        train_softmax = train_softmax.cpu().detach().numpy()
        train_AUROC = roc_auc_score(train_labels, train_softmax[:, 1], average="micro")
        
        valid_loss_list.append(valid_loss.item())
        valid_softmax = valid_softmax.cpu().detach().numpy()
        valid_AUROC = roc_auc_score(valid_labels, valid_softmax[:, 1], average="micro")

        train_AUROC_vec[e] = train_AUROC
        valid_AUROC_vec[e] = valid_AUROC

    train_loss_vec = np.reshape(np.array(train_loss_list), (-1, 1))
    valid_loss_vec = np.reshape(np.array(valid_loss_list), (-1, 1))

    return train_loss_vec, train_AUROC_vec, valid_loss_vec, valid_AUROC_vec

In [50]:
def eval_model_cnn(model, X, y, idx):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X = X.to(device)
    y = y.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(X[idx])
        if isinstance(model, CNN_classification):
            softmax, preds = model.calc_softmax_pred(outputs)
            preds = preds.cpu().numpy()
            softmax = softmax.cpu().numpy()
            labels = y[idx].cpu().numpy()
            AUROC = roc_auc_score(labels, softmax[:, 1])
            precision, recall, _ = precision_recall_curve(labels, softmax[:, 1])
            AUPR = auc(recall, precision)
            return AUROC, AUPR, preds, labels
        else:
            preds = outputs.cpu().numpy()
            labels = y[idx].cpu().numpy()
            pearson = pearsonr(preds, labels)[0]
            return pearson, preds, labels

In [51]:
cell_line = 'E116'
regression_flag = 0
max_epoch = 1000
learning_rate = 1e-4
num_graph_conv_layers = 2
graph_conv_embed_size = 256
num_lin_layers = 3
lin_hidden_size = 256

In [52]:
chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)

if regression_flag == 0:
    num_classes = 2
    task = 'Classification'
else:
    num_classes = 1
    task = 'Regression'


###Initialize start time
start_time = time.time()

today = date.today()
mdy = today.strftime("%Y-%m-%d")
clock = datetime.now()
hms = clock.strftime("%H-%M-%S")
hm = clock.strftime("%Hh-%Mm")
hm_colon = clock.strftime("%H:%M")
date_and_time = mdy + '-at-' + hms


###Test for GPU availability
cuda_flag = torch.cuda.is_available()
if cuda_flag:  
    dev = "cuda" 
else:
    dev = "cpu"  
print(dev) 
device = torch.device(dev)  


###Load input files
base_path = os.getcwd()
save_dir = os.path.join(base_path, 'data', cell_line, 'saved_runs')
hic_sparse_mat_file = os.path.join(base_path, 'data', cell_line, 'hic_sparse.npz')
np_nodes_lab_genes_file = os.path.join(base_path, 'data',  cell_line, \
    'np_nodes_lab_genes_reg' + str(regression_flag) + '.npy')
np_hmods_norm_all_file = os.path.join(base_path, 'data', cell_line, \
    'np_hmods_norm_chip_' + str(chip_res) + 'bp.npy')
df_genes_file = os.path.join(base_path, 'data', cell_line, 'df_genes_reg' + str(regression_flag) + '.pkl')
df_genes = pd.read_pickle(df_genes_file)
    
###Print model specifications
print('Model date and time:')
print(date_and_time, '\n\n')
print('Cell line:', cell_line)
print('Task:', task)
print('ChIP-seq resolution:', str(chip_res))
print('\n')
print('Training set: 70%')
print('Validation set: 15%')
print('Testing set: 15%')
print('\n')
print('Model hyperparameters: ')
print('Number of epochs:', max_epoch)
print('Learning rate:', learning_rate)
print('Number of graph convolutional layers:', str(num_graph_conv_layers))
print('Graph convolutional embedding size:', graph_conv_embed_size)
print('Number of linear layers:', str(num_lin_layers))
print('Linear hidden layer size:', lin_hidden_size)


###Define model inputs
mat = load_npz(hic_sparse_mat_file)
allNodes_hms = np.load(np_hmods_norm_all_file)
hms = allNodes_hms[:, 1:] #only includes features, not node ids
X = torch.tensor(hms).float().reshape(-1, num_feat) 
allNodes = allNodes_hms[:, 0].astype(int)
geneNodes_labs = np.load(np_nodes_lab_genes_file)

geneNodes = geneNodes_labs[:, -2].astype(int)
allLabs = -1*np.ones(np.shape(allNodes))

targetNode_mask = torch.tensor(geneNodes).long()

if regression_flag == 0:
    geneLabs = geneNodes_labs[:, -1].astype(int)
    allLabs[geneNodes] = geneLabs
    Y = torch.tensor(allLabs).long()
else:
    geneLabs = geneNodes_labs[:, -1].astype(float)
    allLabs[geneNodes] = geneLabs
    Y = torch.tensor(allLabs).float()

extract = torch_geometric.utils.from_scipy_sparse_matrix(mat)
data = torch_geometric.data.Data(edge_index = extract[0], edge_attr = extract[1], x = X, y = Y)
G = data


###Define convolutional and linear layer input/output sizes
graph_conv_layer_sizes = [num_feat] + \
    [int(max(graph_conv_embed_size, lin_hidden_size)) \
          for i in np.arange(1, num_graph_conv_layers, 1)] + [lin_hidden_size]
        
lin_hidden_sizes = [graph_conv_layer_sizes[-1]] + \
    [int(max(lin_hidden_size, num_classes)) \
          for i in np.arange(1, num_lin_layers, 1)] + [num_classes]


###Randomize node order and split into 70%/15%/15% training/validation/test sets
pred_idx_shuff = torch.randperm(targetNode_mask.shape[0])

fin_train = np.floor(0.7*pred_idx_shuff.shape[0]).astype(int)
fin_valid = np.floor(0.85*pred_idx_shuff.shape[0]).astype(int)
train_idx = pred_idx_shuff[:fin_train]
valid_idx = pred_idx_shuff[fin_train:fin_valid]
test_idx = pred_idx_shuff[fin_valid:]

train_gene_ID = targetNode_mask[train_idx].numpy()
valid_gene_ID = targetNode_mask[valid_idx].numpy()
test_gene_ID = targetNode_mask[test_idx].numpy()


###Instantiate neural network model, choose optimizer, and print model parameters
if regression_flag == 0:
    model = CNN_classification(num_feat, num_graph_conv_layers, graph_conv_layer_sizes, num_lin_layers, lin_hidden_sizes, num_classes)
else:
    model = CNN_regression(num_feat, num_graph_conv_layers, graph_conv_layer_sizes, num_lin_layers, lin_hidden_sizes, num_classes)

optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, model.parameters()), 
                            lr = learning_rate)

print("\n"+"Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


### For classification:
if regression_flag == 0:
    
    ### Train model
    train_loss_vec, train_AUROC_vec, valid_loss_vec, valid_AUROC_vec = \
        train_model_cnn_classification(model, X, Y, max_epoch, learning_rate, train_idx, valid_idx, optimizer)
    
    ### Evaluate model
    test_AUROC, test_AUPR, test_pred, test_labels, train_AUPR, train_pred, train_labels, \
            valid_AUPR, valid_pred, valid_labels = \
                eval_model_classification(model, G, targetNode_mask, train_idx, valid_idx, test_idx)
    
    ### Save metrics and node predictions
    train_metrics = [train_gene_ID, train_pred, train_labels, train_AUROC_vec, train_AUPR, train_loss_vec]
    np.save(os.path.join(save_dir, 'model_' + date_and_time + '_train_metrics'  + '.npy'), np.asarray(train_metrics, dtype=object))
    
    valid_metrics = [valid_gene_ID, valid_pred, valid_labels, valid_AUROC_vec, valid_AUPR, valid_loss_vec]
    np.save(os.path.join(save_dir, 'model_' + date_and_time + '_valid_metrics'  + '.npy'), np.asarray(valid_metrics, dtype=object))
    
    test_metrics = [test_gene_ID, test_pred, test_labels, test_AUROC, test_AUPR, ['na']]
    np.save(os.path.join(save_dir, 'model_' + date_and_time + '_test_metrics'  + '.npy'), np.asarray(test_metrics, dtype=object))
    
    dataset_list = [train_metrics, valid_metrics, test_metrics]
    df_full_metrics = pd.DataFrame(columns=['Dataset','Node ID','True Label','Predicted Label','Classification'])
    
    for d in np.arange(len(dataset_list)):
        dataset_metrics = dataset_list[d]
        partial_metrics = pd.DataFrame()
    
        partial_metrics['Node ID'] = dataset_metrics[0]
        partial_metrics['True Label'] = dataset_metrics[2]
        partial_metrics['Predicted Label'] = dataset_metrics[1]
        partial_metrics['Classification'] = dataset_metrics[1]*1 + dataset_metrics[2]*2
        partial_metrics['Classification'].replace(to_replace=0, value='TN', inplace=True)
        partial_metrics['Classification'].replace(to_replace=1, value='FP', inplace=True)
        partial_metrics['Classification'].replace(to_replace=2, value='FN', inplace=True)
        partial_metrics['Classification'].replace(to_replace=3, value='TP', inplace=True)
        
        if d == 0:
            partial_metrics['Dataset'] = 'Training'
        elif d == 1:
            partial_metrics['Dataset'] = 'Validation'
        elif d == 2:
            partial_metrics['Dataset'] = 'Testing'
    
        df_full_metrics = pd.concat([df_full_metrics, partial_metrics], ignore_index=True)
    
    df_gene_names = df_genes.iloc[:,:3]
    df_gene_names = df_gene_names.rename(columns={"gene_catalog_name": "ENSEMBL_ID", "abbrev": "Abbreviation",
                                  "hic_node_id" : 'Node ID'})
    df_full_metrics = pd.merge(df_full_metrics, df_gene_names, how='inner', on='Node ID')
    df_full_metrics = df_full_metrics[df_full_metrics.columns[[0,1,5,6,2,3,4]]]

### For regression:
elif regression_flag == 1:

    ### Train model
    train_loss_vec, train_pearson_vec, valid_loss_vec, valid_pearson_vec = \
        train_model_regression(model, G, max_epoch, learning_rate, targetNode_mask, train_idx, valid_idx, optimizer)
    
    ### Evaluate model
    test_pearson, test_pred, test_labels, train_pearson, train_pred, train_labels, \
            valid_pearson, valid_pred, valid_labels = \
                eval_model_regression(model, G, targetNode_mask, train_idx, valid_idx, test_idx)
    
    ### Save metrics and node predictions
    train_metrics = [train_gene_ID, train_pred, train_labels, train_pearson_vec, train_loss_vec]
    np.save(os.path.join(save_dir, 'model_' + date_and_time + '_train_metrics'  + '.npy'), np.asarray(train_metrics, dtype=object))
    
    valid_metrics = [valid_gene_ID, valid_pred, valid_labels, valid_pearson_vec, valid_loss_vec]
    np.save(os.path.join(save_dir, 'model_' + date_and_time + '_valid_metrics'  + '.npy'), np.asarray(valid_metrics, dtype=object))
    
    test_metrics = [test_gene_ID, test_pred, test_labels, test_pearson, ['na']]
    np.save(os.path.join(save_dir, 'model_' + date_and_time + '_test_metrics'  + '.npy'), np.asarray(test_metrics, dtype=object))
    
    dataset_list = [train_metrics, valid_metrics, test_metrics]
    df_full_metrics = pd.DataFrame(columns=['Dataset','Node ID','True Label','Predicted Label'])
    
    for d in np.arange(len(dataset_list)):
        dataset_metrics = dataset_list[d]
        partial_metrics = pd.DataFrame()
    
        partial_metrics['Node ID'] = dataset_metrics[0]
        partial_metrics['True Label'] = dataset_metrics[2]
        partial_metrics['Predicted Label'] = dataset_metrics[1]
        
        if d == 0:
            partial_metrics['Dataset'] = 'Training'
        elif d == 1:
            partial_metrics['Dataset'] = 'Validation'
        elif d == 2:
            partial_metrics['Dataset'] = 'Testing'
    
        df_full_metrics = pd.concat([df_full_metrics, partial_metrics], ignore_index=True)
    
    df_gene_names = df_genes.iloc[:,:3]
    df_gene_names = df_gene_names.rename(columns={"gene_catalog_name": "ENSEMBL_ID", "abbrev": "Abbreviation",
                                  "hic_node_id" : 'Node ID'})
    df_full_metrics = pd.merge(df_full_metrics, df_gene_names, how='inner', on='Node ID')
    df_full_metrics = df_full_metrics[df_full_metrics.columns[[0,1,4,5,2,3]]]


### Print elapsed time and performance
elapsed = (time.time() - start_time)
elapsed_h = int(elapsed//3600)
elapsed_m = int((elapsed - elapsed_h*3600)//60)
elapsed_s = int(elapsed - elapsed_h*3600 - elapsed_m*60)
print('Elapsed time: {0:02d}:{1:02d}:{2:02d}'.format(elapsed_h, elapsed_m, elapsed_s))

print('\nPerformance:')
if regression_flag == 0:
    print('Test AUROC:', test_AUROC, '\n')
    print('Test AUPR:', test_AUPR, '\n\n')
elif regression_flag == 1:
    print('Test pearson:', test_pearson, '\n')

cuda
Model date and time:
2024-11-01-at-10-57-20 


Cell line: E116
Task: Classification
ChIP-seq resolution: 10000


Training set: 70%
Validation set: 15%
Testing set: 15%


Model hyperparameters: 
Number of epochs: 1000
Learning rate: 0.0001
Number of graph convolutional layers: 2
Graph convolutional embedding size: 256
Number of linear layers: 3
Linear hidden layer size: 256

Model's state_dict:
conv1.weight 	 torch.Size([6, 6, 3])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([256, 6, 3])
conv2.bias 	 torch.Size([256])
lin1.weight 	 torch.Size([256, 256])
lin1.bias 	 torch.Size([256])
lin2.weight 	 torch.Size([256, 256])
lin2.bias 	 torch.Size([256])
lin3.weight 	 torch.Size([2, 256])
lin3.bias 	 torch.Size([2])


Epoch 0 out of 1000


NameError: name 'edge_index' is not defined